<a href="https://colab.research.google.com/github/immanuelraju/CARTOONIFY/blob/main/my%20project%20python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pymupdf
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 25.2 MB/s eta 0:00:00


In [2]:
# Step 2: Import libraries
import fitz  # PyMuPDF for PDF extraction
import re
from transformers import pipeline
import pandas as pd

# Step 3: Define a function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    """
    This function extracts text from each page of the PDF document
    and combines it into a single string for further processing.
    """
    text = ""
    doc = fitz.open(pdf_path)
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        text += page.get_text("text")
    doc.close()
    return text

# Step 4: Define a function to segment text into meaningful sections
def segment_text(text):
    """
    This function segments the text into sections based on common patterns in financial reports.
    It returns a list of section strings.
    """
    # Split the text based on section headers and bullet points
    sections = re.split(r"\n\s*\d+\.\s|\n[A-Z]+\s*\n", text)
    return [section.strip() for section in sections if len(section.strip()) > 50]

# Step 5: Initialize the Hugging Face summarizer pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Step 6: Define a function to extract and summarize key insights
def extract_key_insights(text_segments):
    """
    This function uses keyword-based filtering to find segments related to key topics,
    and then summarizes those segments for concise insights.
    """
    key_info = []
    # Define investor-relevant keywords
    keywords = ["growth", "revenue", "prospects", "risk", "earnings", "business change", "trigger"]

    # Iterate through each segment and summarize if it contains keywords
    for segment in text_segments:
        if any(keyword in segment.lower() for keyword in keywords):
            # Summarize the section using the pre-trained model
            summary = summarizer(segment, max_length=100, min_length=30, do_sample=False)
            key_info.append(summary[0]['summary_text'])

    return key_info

# Step 7: Run the extraction and summarization on the PDF
# Mount Google Drive to access the PDF if it's stored there
from google.colab import drive
drive.mount('/content/drive')

# Path to the PDF document (update this path as needed)
pdf_path = "/content/drive/MyDrive/InvestorReport.pdf"

# Extract and process the PDF text
text = extract_text_from_pdf(pdf_path)
segments = segment_text(text)
insights = extract_key_insights(segments)

# Step 8: Display the extracted insights
print("Key Insights for Investor Evaluation:")
for idx, insight in enumerate(insights, start=1):
    print(f"Insight {idx}: {insight}")

# (Optional) Save insights to a CSV file for easy download and review
df = pd.DataFrame({"Insights": insights})
df.to_csv("/content/investor_key_insights.csv", index=False)
print("Insights saved to 'investor_key_insights.csv'")

# Step 9: Optional download link for the CSV file
from google.colab import files
files.download("/content/investor_key_insights.csv")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


ValueError: mount failed